## Problem Background
Lagos is a big city with big problem, but there's no coordinate on the internet for Lagos suburbs and neighborhood.
This will be solved in this project.
Secondly, I want to investigate the top businesses in a Lagos Borough, my pick is Alimosho Local GOvernemt, the
biggest Local Government in Lagos State, see which business is more popular, and which is less popular.

The output from this data would show the common places available in the LGA, and those that may need to be invested it, coming from lack of competition for the business we have in mind.

## Description of the Data
No Data is available on the internet for this, so I will have to construct one through data wrangling multiple sources
I was able to get the Local Goverment Areas(LGAs), and their wards, so I will be using GeoCoder to get their 
various coordinates. The coordinates will be used to map the LGA, and then try to get the most popular venues and places
 in the LGA. If time permits, I will look at the population and its effect on voting pattern. Just maybe

In [72]:
import pandas as pd

### Getting the data need for this assignment from the internet

In [2]:
d=pd.read_html('https://en.wikipedia.org/wiki/Lagos_State')

#### There were about 6 tables downloaded, I assigned the required one

As you can see, it's raw data, without coordinates.
Just the Local Government Area, and there Wards

In [3]:
dfr=d[4]
dfr

,LGA,Wards
0,Agege,Isale/Idimangoro; Iloro/Onipetesi; Oniwaya/Pap...
1,Ajeromi/Ifelodun,Ago Hausa; Awodi-Ora; Wilmer; Olodi; Tolu; Tem...
2,Alimosho,Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri...
3,Amuwo-Odofin,"Amuwo-Odofin Housing Estate, Mile 2; Festac 1;..."
4,Apapa,Apapa I (Marine Rd. and environs); Apapa II (L...
5,Badagry,Posukoh; Awhanjigoh; Ibereko; Keta-East; Iworo...
6,Epe,Etita/Ebode; Lagbade; Popo-Oba; Oke-Balogun; A...
7,Eti-Osa,Victoria Island I; Victoria Island II; Ilasan ...
8,Ibeju/Lekki,"Ibeju I; N2, (Ibeju II); Orimedu I; 02, (Orime..."
9,Ifako-Ijaye,Ijaye; Old Ifako/Karaole; New Ifako/Oyemekun; ...


### Import the Needed Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


#### To Use the data, the Local Government Areas (LGA) are unknown, I added a popular city name to help the geocoder

In [5]:
dfr['Locale']=dfr.LGA + (', Lagos, Nigeria')
dfr

,LGA,Wards,Locale
0,Agege,Isale/Idimangoro; Iloro/Onipetesi; Oniwaya/Pap...,"Agege, Lagos, Nigeria"
1,Ajeromi/Ifelodun,Ago Hausa; Awodi-Ora; Wilmer; Olodi; Tolu; Tem...,"Ajeromi/Ifelodun, Lagos, Nigeria"
2,Alimosho,Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri...,"Alimosho, Lagos, Nigeria"
3,Amuwo-Odofin,"Amuwo-Odofin Housing Estate, Mile 2; Festac 1;...","Amuwo-Odofin, Lagos, Nigeria"
4,Apapa,Apapa I (Marine Rd. and environs); Apapa II (L...,"Apapa, Lagos, Nigeria"
5,Badagry,Posukoh; Awhanjigoh; Ibereko; Keta-East; Iworo...,"Badagry, Lagos, Nigeria"
6,Epe,Etita/Ebode; Lagbade; Popo-Oba; Oke-Balogun; A...,"Epe, Lagos, Nigeria"
7,Eti-Osa,Victoria Island I; Victoria Island II; Ilasan ...,"Eti-Osa, Lagos, Nigeria"
8,Ibeju/Lekki,"Ibeju I; N2, (Ibeju II); Orimedu I; 02, (Orime...","Ibeju/Lekki, Lagos, Nigeria"
9,Ifako-Ijaye,Ijaye; Old Ifako/Karaole; New Ifako/Oyemekun; ...,"Ifako-Ijaye, Lagos, Nigeria"


### Loop through the DataFrame to get the Coordinates. (Latitude and Longitude)

In [8]:
from geopy.exc import GeocoderTimedOut
for cord in (dfr.index):
    geolocator = Nominatim(user_agent="lag_explore")
    location = geolocator.geocode(dfr['Locale'][cord], timeout=20000)
    
    if location:
        latitude= location.latitude
        longitude= location.longitude
    else:
        latitude= None
        longitude= None
        
    dfr['Latitude'][cord]=latitude
    dfr['Longitude'][cord]=longitude
    

dfr

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LGA,Wards,Locale,Latitude,Longitude
0,Agege,Isale/Idimangoro; Iloro/Onipetesi; Oniwaya/Pap...,"Agege, Lagos, Nigeria",6.625256,3.311209
1,Ajeromi/Ifelodun,Ago Hausa; Awodi-Ora; Wilmer; Olodi; Tolu; Tem...,"Ajeromi/Ifelodun, Lagos, Nigeria",6.455122,3.335946
2,Alimosho,Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri...,"Alimosho, Lagos, Nigeria",6.584343,3.257631
3,Amuwo-Odofin,"Amuwo-Odofin Housing Estate, Mile 2; Festac 1;...","Amuwo-Odofin, Lagos, Nigeria",6.447023,3.266280
4,Apapa,Apapa I (Marine Rd. and environs); Apapa II (L...,"Apapa, Lagos, Nigeria",6.445187,3.368373
5,Badagry,Posukoh; Awhanjigoh; Ibereko; Keta-East; Iworo...,"Badagry, Lagos, Nigeria",6.438730,2.907486
6,Epe,Etita/Ebode; Lagbade; Popo-Oba; Oke-Balogun; A...,"Epe, Lagos, Nigeria",6.524750,4.057246
7,Eti-Osa,Victoria Island I; Victoria Island II; Ilasan ...,"Eti-Osa, Lagos, Nigeria",6.460231,3.532181
8,Ibeju/Lekki,"Ibeju I; N2, (Ibeju II); Orimedu I; 02, (Orime...","Ibeju/Lekki, Lagos, Nigeria",6.471125,3.814750
9,Ifako-Ijaye,Ijaye; Old Ifako/Karaole; New Ifako/Oyemekun; ...,"Ifako-Ijaye, Lagos, Nigeria",6.660436,3.321539


#### Saved the New DataFrame to a CSV File (Optional: because I work from multiple systems)

In [14]:
dfr.to_csv(r'C:\Users\Joshy\Downloads\LGAandWardsCoords.csv')

#### Renamed the Columns for easy reading to everyone

In [15]:
dfr.rename(columns={'LGA':'Borough', 'Wards':'Neighborhood'}, inplace=True)
dfr

,Borough,Neighborhood,Locale,Latitude,Longitude
0,Agege,Isale/Idimangoro; Iloro/Onipetesi; Oniwaya/Pap...,"Agege, Lagos, Nigeria",6.625256,3.311209
1,Ajeromi/Ifelodun,Ago Hausa; Awodi-Ora; Wilmer; Olodi; Tolu; Tem...,"Ajeromi/Ifelodun, Lagos, Nigeria",6.455122,3.335946
2,Alimosho,Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri...,"Alimosho, Lagos, Nigeria",6.584343,3.257631
3,Amuwo-Odofin,"Amuwo-Odofin Housing Estate, Mile 2; Festac 1;...","Amuwo-Odofin, Lagos, Nigeria",6.447023,3.266280
4,Apapa,Apapa I (Marine Rd. and environs); Apapa II (L...,"Apapa, Lagos, Nigeria",6.445187,3.368373
5,Badagry,Posukoh; Awhanjigoh; Ibereko; Keta-East; Iworo...,"Badagry, Lagos, Nigeria",6.438730,2.907486
6,Epe,Etita/Ebode; Lagbade; Popo-Oba; Oke-Balogun; A...,"Epe, Lagos, Nigeria",6.524750,4.057246
7,Eti-Osa,Victoria Island I; Victoria Island II; Ilasan ...,"Eti-Osa, Lagos, Nigeria",6.460231,3.532181
8,Ibeju/Lekki,"Ibeju I; N2, (Ibeju II); Orimedu I; 02, (Orime...","Ibeju/Lekki, Lagos, Nigeria",6.471125,3.814750
9,Ifako-Ijaye,Ijaye; Old Ifako/Karaole; New Ifako/Oyemekun; ...,"Ifako-Ijaye, Lagos, Nigeria",6.660436,3.321539


### Get the Lagos, Nigeria Coordinate

In [16]:
address = 'Lagos, Nigeria'

geolocator = Nominatim(user_agent="lagos_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lagos, Nigeria are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lagos, Nigeria are 6.4550575, 3.3941795.


#### Plot the Map, and Add a Marker

In [17]:
lagosmap=folium.Map(location=[latitude,longitude], zoom_start=10)
lagosmap

In [18]:
for lat, lng, borough, neighborhood in zip(dfr['Latitude'], dfr['Longitude'], dfr['Borough'], dfr['Neighborhood']):
    label='{},{}'.format(neighborhood, borough)
    label=folium.Popup(label, perse_html=True)
    folium.CircleMarker([lat,lng], radius=5, 
                        popup=label, color='red',
                         fill=True, fill_color='red', 
                        fill_opacity=0.7,
                        parse_html=False).add_to(lagosmap)
    
lagosmap


#### Concentration on a Borough = **Alimosho**

In [19]:
alimosho_lga=dfr[dfr['Borough']=='Alimosho'].reset_index(drop=True)
alimosho_lga.head()

,Borough,Neighborhood,Locale,Latitude,Longitude
0,Alimosho,Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri...,"Alimosho, Lagos, Nigeria",6.584343,3.257631


In [21]:
address = 'Alimosho, Lagos, Nigeria'

geolocator = Nominatim(user_agent="lagos_explorer")
location = geolocator.geocode(address)
alatitude = location.latitude
alongitude = location.longitude
print('The geograpical coordinate of Alimosho are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Alimosho are 6.4550575, 3.3941795.


In [22]:
alimosho_map=folium.Map(location=[alatitude,alongitude], zoom_start=13)

for lat, lng, label in zip(alimosho_lga['Latitude'], alimosho_lga['Longitude'], alimosho_lga['Neighborhood']):
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
           [lat,lng],
            radius=5, popup=label,
            color='blue', fill_color = 'blue',
            fill=True, fill_opacity=0.7, parse_html=False).add_to(alimosho_map)
    
    
alimosho_map

#### Define Foursquare Credentials

In [23]:
import re
CLIENT_ID='KTE4N43UQ1MLSEA3LZ0DENQQMIKNMMF33MEHQ12RYWNRLY4O'
CLIENT_SECRET='KKGIGI2B4HK4UR1FYJRK00NOOWUICD4Y1WPTDZRWRJ0ICGLC'
VERSION='20200228'


### Now Settling into the Borough I selected for Assignment
###### I grew up in this Borough, and know the Neighborhoods very well.
###### If I planned on setting up a business, this would be the first Borough, I'd think of.

In [24]:
cols=['Neighs', 'Latitudes', 'Longitudes']
alimhoods=pd.DataFrame(columns=cols)
alimhoods

,Neighs,Latitudes,Longitudes


In [25]:
alimhoods_data=alimosho_lga.loc[0, 'Neighborhood']
alimhoods_data

'Shasha/Akowonjo; Egbeda/Alimosho; Idimu/Isheri Olofin; Akesan; Ikotun/Ijegun; Egbe/Agodo; Igando/Egan; Ipaja North; Ipaja South; Ayobo/Ijon Village (Camp David); Pleasure/Oke-Odo; Abule-Egba/Aboru/Meiran/Alagbado'

In [26]:
alimhoods['Neighs'] = re.split('; |/', alimhoods_data)

### Broke the Borough Down into the Neighboorhoods, we have 23 Neighboorhoods here
##### It's the Biggest Borough in Lagos, Nigeria, and the by far the most Populous 

In [29]:
alimhoods

,Neighs,Latitudes,Longitudes
0,Shasha,NaN,NaN
1,Akowonjo,NaN,NaN
2,Egbeda,NaN,NaN
3,Alimosho,NaN,NaN
4,Idimu,NaN,NaN
5,Isheri Olofin,NaN,NaN
6,Akesan,NaN,NaN
7,Ikotun,NaN,NaN
8,Ijegun,NaN,NaN
9,Egbe,NaN,NaN


In [30]:
from geopy.exc import GeocoderTimedOut
for i in (alimhoods.index):
    geolocator = Nominatim(user_agent="ali_explore")
    location = geolocator.geocode(alimhoods['Neighs'][i], timeout=20000)
    
    if location:
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude= None
        longitude= None
        
    alimhoods['Latitudes'][i]=latitude
    alimhoods['Longitudes'][i]=longitude
    

alimhoods

,Neighs,Latitudes,Longitudes
0,Shasha,6.91272,4.34365
1,Akowonjo,6.61048,3.31031
2,Egbeda,5.23472,6.7525
3,Alimosho,6.58434,3.25763
4,Idimu,6.57224,3.25704
5,Isheri Olofin,6.57611,3.27356
6,Akesan,6.54747,3.2324
7,Ikotun,8.22871,4.61474
8,Ijegun,6.51616,3.25487
9,Egbe,52.3699,-1.48129


#### Remove inconsistencies 

In [32]:
calm=alimhoods.replace(to_replace='None', value=np.nan).dropna()
alimhoods

,Neighs,Latitudes,Longitudes
0,Shasha,6.91272,4.34365
1,Akowonjo,6.61048,3.31031
2,Egbeda,5.23472,6.7525
3,Alimosho,6.58434,3.25763
4,Idimu,6.57224,3.25704
5,Isheri Olofin,6.57611,3.27356
6,Akesan,6.54747,3.2324
7,Ikotun,8.22871,4.61474
8,Ijegun,6.51616,3.25487
9,Egbe,52.3699,-1.48129


In [33]:
calm.rename(columns={'Neighs':'Neighborhood'}, inplace=True)

In [34]:
calm

,Neighborhood,Latitudes,Longitudes
0,Shasha,6.91272,4.34365
1,Akowonjo,6.61048,3.31031
2,Egbeda,5.23472,6.7525
3,Alimosho,6.58434,3.25763
4,Idimu,6.57224,3.25704
5,Isheri Olofin,6.57611,3.27356
6,Akesan,6.54747,3.2324
7,Ikotun,8.22871,4.61474
8,Ijegun,6.51616,3.25487
9,Egbe,52.3699,-1.48129


In [35]:
alimhoods

,Neighs,Latitudes,Longitudes
0,Shasha,6.91272,4.34365
1,Akowonjo,6.61048,3.31031
2,Egbeda,5.23472,6.7525
3,Alimosho,6.58434,3.25763
4,Idimu,6.57224,3.25704
5,Isheri Olofin,6.57611,3.27356
6,Akesan,6.54747,3.2324
7,Ikotun,8.22871,4.61474
8,Ijegun,6.51616,3.25487
9,Egbe,52.3699,-1.48129


In [38]:
nei_lat=calm.loc[1, 'Latitudes']
nei_lng=calm.loc[1, 'Longitudes']
nei_name=calm.loc[1,'Neighborhood']

print(nei_name, 'is in coordinate ', nei_lat, ',',nei_lng)

Akowonjo is in coordinate  6.6104787 , 3.3103079


### Getting the URL

In [39]:
radius=5000
LIMIT=100
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET, VERSION, nei_lat, nei_lng, radius, LIMIT)

In [40]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7cec8bb9a389001bf68db8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Lagos',
  'headerFullLocation': 'Lagos',
  'headerLocationGranularity': 'city',
  'totalResults': 39,
  'suggestedBounds': {'ne': {'lat': 6.655478745000045,
    'lng': 3.355524537252914},
   'sw': {'lat': 6.565478654999954, 'lng': 3.2650912627470863}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '502806dce4b0f23b021f3b77',
       'name': 'KFC',
       'location': {'lat': 6.604589745106469,
        'lng': 3.3089358809010045,
        'labeledLatLngs': [{'label': 'display',
          'lat': 6.604589745106469,
          'lng': 3.3089358809010045}],
        'distance': 672

In [41]:
def getCAT(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
        
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']


In [42]:
venues=results['response']['groups'][0]['items']

nearby_venues=pd.json_normalize(venues)

filtered_cols=['venue.name', 'venue.catergories', 'venue.location.lat', 'venue.location.lng']

#nearby_venues= nearby_venues.loc[: , filtered_cols]
nearby_venues['venue.categories']=nearby_venues.apply(getCAT, axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,referralId,count,items,id,name,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,categories,count,groups,address,crossStreet,neighborhood,postalCode,id
0,e-0-502806dce4b0f23b021f3b77-0,0,"[{'summary': 'This spot is popular', 'type': '...",502806dce4b0f23b021f3b77,KFC,6.604590,3.308936,"[{'label': 'display', 'lat': 6.604589745106469...",672,NG,Egbeda,Lagos,Nigeria,"[Egbeda, Lagos, Nigeria]",Fast Food Restaurant,0,[],NaN,NaN,NaN,NaN,NaN
1,e-0-4e8f081c77c843122cdaa1ad-1,0,"[{'summary': 'This spot is popular', 'type': '...",4e8f081c77c843122cdaa1ad,Pink Ice Boutique,6.601810,3.301896,"[{'label': 'display', 'lat': 6.601810454245876...",1340,NG,Egbeda-Akowonjo,Lagos,Nigeria,"[117, Akowonjo Road (Jimoh Bus Stop), Egbeda-A...",Boutique,0,[],"117, Akowonjo Road",Jimoh Bus Stop,NaN,NaN,NaN
2,e-0-5205f22911d2055531e5599b-2,0,"[{'summary': 'This spot is popular', 'type': '...",5205f22911d2055531e5599b,"National Stadium, Surulere",6.593562,3.304300,"[{'label': 'display', 'lat': 6.59356167915684,...",1996,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos, Nigeria]",Athletics & Sports,0,[],NaN,NaN,NaN,NaN,NaN
3,e-0-4f75a8e7e4b06c81e0e3ba05-3,0,"[{'summary': 'This spot is popular', 'type': '...",4f75a8e7e4b06c81e0e3ba05,Wings Restaurant,6.576023,3.325612,"[{'label': 'display', 'lat': 6.576022675981682...",4192,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos, Nigeria]",Restaurant,0,[],NaN,NaN,NaN,NaN,NaN
4,e-0-4f6cbb4be4b0861075eb93bf-4,0,"[{'summary': 'This spot is popular', 'type': '...",4f6cbb4be4b0861075eb93bf,"ICE CREAM PLUS, Medical Road, Ikeja.",6.596598,3.340001,"[{'label': 'display', 'lat': 6.596598066134291...",3628,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos, Nigeria]",Soup Place,0,[],NaN,NaN,NaN,NaN,NaN


In [43]:
nearby_venues.shape

(39, 22)

#### Exploring My Neighborhood - Alimosho

In [44]:
def getVenues(names, latitudes, longitudes, radius=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        
        results=requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
                    name, lat, lng, v['venue']['name'],
                    v['venue']['location']['lat'],
                    v['venue']['location']['lng'],
                    v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns=['Neigborhoods', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

#### Creating a new dataFrame for the spots in Alimosho

In [46]:
akowonjo_venues=getVenues(names=calm['Neighborhood'], latitudes=calm['Latitudes'], longitudes=calm['Longitudes'])

Shasha
Akowonjo
Egbeda
Alimosho
Idimu
Isheri Olofin
Akesan
Ikotun
Ijegun
Egbe
Agodo
Igando
Egan
Ipaja North
Ipaja South
Ayobo
Pleasure
Oke-Odo
Abule-Egba
Aboru
Meiran
Alagbado


In [47]:
akowonjo_venues.head()

,Neigborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akowonjo,6.610479,3.310308,KFC,6.604590,3.308936,Fast Food Restaurant
1,Akowonjo,6.610479,3.310308,Dopemu Under Bridge,6.608221,3.310003,Bus Station
2,Akowonjo,6.610479,3.310308,MTN Connect Store,6.604171,3.308110,Mobile Phone Shop
3,Akowonjo,6.610479,3.310308,Cement Bus Stop,6.607652,3.318337,Bus Station
4,Akowonjo,6.610479,3.310308,Mango Busstop,6.606112,3.317628,Bus Station


In [48]:
akowonjo_venues.groupby('Neigborhoods').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neigborhoods,,,,,,
Akesan,1,1,1,1,1,1
Akowonjo,6,6,6,6,6,6
Egan,1,1,1,1,1,1
Egbe,5,5,5,5,5,5
Idimu,1,1,1,1,1,1
Ipaja North,4,4,4,4,4,4
Isheri Olofin,5,5,5,5,5,5
Pleasure,22,22,22,22,22,22


In [49]:
print('There are ',len(akowonjo_venues['Venue Category'].unique()),' unique catergories in Akowonjo Area')

There are  25  unique catergories in Akowonjo Area


#### Analysing Each Neighborhood

In [50]:
akowonjo_onehot=pd.get_dummies(akowonjo_venues[['Venue Category']], prefix="", prefix_sep="")
akowonjo_onehot['Neighborhood']=akowonjo_venues['Neigborhoods']

fixed_columns=[akowonjo_onehot.columns[-1]] + list(akowonjo_onehot.columns[:-1])
akowonjo_onehot=akowonjo_onehot[fixed_columns]
akowonjo_onehot

,Neighborhood,Airport,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Furniture / Home Store,Grocery Store,Harbor / Marina,Hotel,Indian Restaurant,Lighthouse,Market,Mobile Phone Shop,Museum,Nightclub,Optical Shop,Park,Pub,Restaurant,Shopping Mall
0,Akowonjo,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Akowonjo,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Akowonjo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Akowonjo,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Akowonjo,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Akowonjo,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Idimu,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Isheri Olofin,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,Isheri Olofin,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Isheri Olofin,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [51]:
akowonjo_onehot.shape

(45, 26)

In [52]:
akowonjo_grouped=akowonjo_onehot.groupby('Neighborhood').mean().reset_index()
akowonjo_grouped

,Neighborhood,Airport,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Discount Store,Fast Food Restaurant,Furniture / Home Store,Grocery Store,Harbor / Marina,Hotel,Indian Restaurant,Lighthouse,Market,Mobile Phone Shop,Museum,Nightclub,Optical Shop,Park,Pub,Restaurant,Shopping Mall
0,Akesan,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Akowonjo,0.0,0.0,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.00,0.00,0.000000,0.0,0.00,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Egan,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
3,Egbe,0.2,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.000000,0.00,0.20,0.000000,0.0,0.00,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Idimu,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Ipaja North,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.25,0.25,0.000000,0.0,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Isheri Olofin,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.00,0.40,0.000000,0.2,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Pleasure,0.0,0.0,0.045455,0.045455,0.045455,0.045455,0.045455,0.090909,0.090909,0.045455,0.0,0.045455,0.00,0.00,0.090909,0.0,0.00,0.000000,0.0,0.045455,0.045455,0.090909,0.181818,0.0,0.045455


In [53]:
akowonjo_grouped.shape

(8, 26)

### Get the top 5 Venues

In [ ]:
top_venues = 5

for hood in akowonjo_grouped['Neighborhood']:
    print("-----"+hood+"-----")
    temp=akowonjo_grouped[akowonjo_grouped['Neighborhood']== hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq' :2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

In [55]:
def hotspot_venues(row, top_venues):
    row_categories=row.iloc[1:]
    row_sorted=row_categories.sort_values(ascending=False)
    
    return row_sorted.index.values[0:top_venues]

#### Top Ten Places in Alimosho Local Governemnt Area and Neighborhood

In [59]:
num_top =10
indicators=['st', 'nd', 'rd']
columns=['Neighborhood']

for ind in np.arange(num_top):
    try:
        columns.append('{}{} Most Common Venue'.format (ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = akowonjo_grouped['Neighborhood']

for ind in np.arange(akowonjo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = hotspot_venues(akowonjo_grouped.iloc[ind, :], num_top)


neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akesan,Convenience Store,Shopping Mall,Grocery Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Discount Store
1,Akowonjo,Bus Station,Mobile Phone Shop,Fast Food Restaurant,Shopping Mall,Grocery Store,Bar,Café,Chinese Restaurant,Clothing Store,Coffee Shop
2,Egan,Restaurant,Shopping Mall,Grocery Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store
3,Egbe,Airport,Chinese Restaurant,Museum,Convenience Store,Hotel,Furniture / Home Store,Bar,Bus Station,Café,Clothing Store
4,Idimu,Bar,Shopping Mall,Grocery Store,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Discount Store
5,Ipaja North,Harbor / Marina,Market,Convenience Store,Hotel,Furniture / Home Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store
6,Isheri Olofin,Hotel,Furniture / Home Store,Bar,Lighthouse,Shopping Mall,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop
7,Pleasure,Pub,Discount Store,Park,Indian Restaurant,Convenience Store,Shopping Mall,Optical Shop,Nightclub,Bus Station,Café


##### Cluster the Neighborhood

In [60]:
kluster=5

akowonjo_cluster=akowonjo_grouped.drop('Neighborhood',1)

kmeans=KMeans(n_clusters=kluster, random_state=0).fit(akowonjo_cluster)
kmeans.labels_[0:10]

array([4, 1, 3, 2, 0, 2, 2, 2])

In [61]:
neighborhoods_venues_sorted.insert(0, 'Avan', kmeans.labels_)

akowonjo_merged = calm

akowonjo_merged = akowonjo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

akowonjo_merged

,Neighborhood,Latitudes,Longitudes,Avan,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Shasha,6.91272,4.34365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akowonjo,6.61048,3.31031,1.0,Bus Station,Mobile Phone Shop,Fast Food Restaurant,Shopping Mall,Grocery Store,Bar,Café,Chinese Restaurant,Clothing Store,Coffee Shop
2,Egbeda,5.23472,6.7525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alimosho,6.58434,3.25763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Idimu,6.57224,3.25704,0.0,Bar,Shopping Mall,Grocery Store,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Discount Store
5,Isheri Olofin,6.57611,3.27356,2.0,Hotel,Furniture / Home Store,Bar,Lighthouse,Shopping Mall,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop
6,Akesan,6.54747,3.2324,4.0,Convenience Store,Shopping Mall,Grocery Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Discount Store
7,Ikotun,8.22871,4.61474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ijegun,6.51616,3.25487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Egbe,52.3699,-1.48129,2.0,Airport,Chinese Restaurant,Museum,Convenience Store,Hotel,Furniture / Home Store,Bar,Bus Station,Café,Clothing Store


In [62]:
latitude=6.584343
longitude=3.257631

In [64]:

map_clusters=folium.Map(location=[6.584343, 3.257631], zoom_start=12)

x= np.arange(kluster)
ys=[i+x+(i*x)**2 for i in range(kluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors =[]
for lat, lon, poi, cluster in zip(akowonjo_merged['Latitudes'], akowonjo_merged['Longitudes'], akowonjo_merged['Neighborhood'], akowonjo_merged['Avan']):
    label=folium.Popup(str(poi) + 'Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
            [lat,lon], radius=5, popup=label,
            fill=True, fill_opacity=0.7).add_to(map_clusters)
    

    
    
map_clusters

#### Examining the Clusters

In [66]:
akowonjo_merged.loc[akowonjo_merged['Avan'] == 0, akowonjo_merged.columns[[1]+list(range(5, akowonjo_merged.shape[1]))]]

,Latitudes,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,6.57224,Shopping Mall,Grocery Store,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Discount Store


In [68]:
akowonjo_merged.loc[akowonjo_merged['Avan'] == 1, akowonjo_merged.columns[[1]+list(range(5, akowonjo_merged.shape[1]))]]

,Latitudes,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,6.61048,Mobile Phone Shop,Fast Food Restaurant,Shopping Mall,Grocery Store,Bar,Café,Chinese Restaurant,Clothing Store,Coffee Shop


In [69]:
akowonjo_merged.loc[akowonjo_merged['Avan'] == 2, akowonjo_merged.columns[[1]+list(range(5, akowonjo_merged.shape[1]))]]

,Latitudes,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,6.57611,Furniture / Home Store,Bar,Lighthouse,Shopping Mall,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop
9,52.3699,Chinese Restaurant,Museum,Convenience Store,Hotel,Furniture / Home Store,Bar,Bus Station,Café,Clothing Store
13,6.60883,Market,Convenience Store,Hotel,Furniture / Home Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store
17,52.7144,Discount Store,Park,Indian Restaurant,Convenience Store,Shopping Mall,Optical Shop,Nightclub,Bus Station,Café


In [70]:
akowonjo_merged.loc[akowonjo_merged['Avan'] == 3, akowonjo_merged.columns[[1]+list(range(5, akowonjo_merged.shape[1]))]]

,Latitudes,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,43.9991,Shopping Mall,Grocery Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store


In [71]:
akowonjo_merged.loc[akowonjo_merged['Avan'] == 4, akowonjo_merged.columns[[1]+list(range(5, akowonjo_merged.shape[1]))]]

,Latitudes,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,6.54747,Shopping Mall,Grocery Store,Bar,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Discount Store


##### The End!!!

###### So In the end, BookStores and Clinics (Hospitals) are not part of the top 10 in the area